# Mobile Get Input Notebook - Phase 2: Batch Point Generation

**Phase 2**: Generate all receiver points at once using radial distribution.

This phase creates uniformly distributed receiver points around the transmitter at multiple distances and azimuths, outputting a GeoDataFrame ready for Phase 3.

## Prerequisites
- Run **Phase 0** first (phase0_setup.ipynb)
- Run **Phase 1** first (phase1_data_prep.ipynb) - optional but recommended

## Workflow
1. **Import Phase 0 outputs**: CONFIG, transmitter, paths
2. **Generate receivers**: Create points at all distances × all azimuths
3. **GeoDataFrame**: Store geometry + metadata (distance, azimuth)
4. **Output**: GeoDataFrame with ~13k points ready for Phase 3

## Output
- GeoDataFrame with receiver points (geometry + distance + azimuth)
- ~13,000 points (22 distances × 36 azimuths + 1 transmitter point)
- Stored in `receivers_gdf` variable for Phase 3

## Setup: Import from Phase 0

This cell imports all the setup from Phase 0.

In [34]:
# Import everything from Phase 0 using %run magic
%run phase0_setup.ipynb

print("\n✓ Phase 0 setup imported successfully")
print(f"  Transmitter: {tx}")
print(f"  Project root: {project_root}")

✓ All imports successful
Project root: /Users/oz/Documents/mst_gis
✓ All data directories created
  profiles: /Users/oz/Documents/mst_gis/data/input/profiles
  api_data: /Users/oz/Documents/mst_gis/data/intermediate/api_data
  reference: /Users/oz/Documents/mst_gis/data/input/reference
  output: /Users/oz/Documents/mst_gis/data/output/spreadsheets
✓ Loaded configuration from config_example.json
Transmitter: (9.345, -13.40694)
Azimuths: 36 | Profile points: 366
Frequency: 0.9 GHz | Polarization: 1

✓ Transmitter created:
  Transmitter(tx_id='TX_0001', lon=-13.40694, lat=9.345, htg=57, f=0.9, pol=1, p=50, hrg=10)

Initializing SRTM elevation data...
✓ SRTM elevation data ready (0.01s)
  TX elevation: 13m
  Cache location: /Users/oz/Documents/mst_gis/data/intermediate/elevation_cache

  Loading HGT tile into memory for Phase 3...
  ✓ HGT loaded: (1201, 1201) array, dtype=int16

✓ Phase 0 setup imported successfully
  Transmitter: Transmitter(tx_id='TX_0001', lon=-13.40694, lat=9.345, htg=

## Helper Function: Generate Receivers

Create uniformly distributed receiver points at multiple distances and azimuths.

In [35]:
def generate_receivers_radial_multi(
    tx,
    distances_km,
    azimuths_deg,
    include_tx_point=False,
):
    """Generate receivers on multiple rings around transmitter.
    
    Args:
        tx: Transmitter object with lon, lat, tx_id
        distances_km: Array/list of distances in km
        azimuths_deg: Array/list of azimuths in degrees (0-360)
        include_tx_point: If True, include transmitter as rx_id=0
    
    Returns:
        GeoDataFrame with columns: tx_id, rx_id, distance_km, azimuth_deg, geometry
    """
    # Create transmitter point and get UTM CRS
    tx_gdf = gpd.GeoDataFrame(
        {"tx_id": [tx.tx_id]},
        geometry=[Point(tx.lon, tx.lat)],
        crs="EPSG:4326",
    )
    utm_crs = tx_gdf.estimate_utm_crs()
    tx_utm = tx_gdf.to_crs(utm_crs)
    tx_pt = tx_utm.geometry.iloc[0]

    rows = []
    rx_id = 1

    # Optional: add transmitter point at distance=0
    if include_tx_point:
        rows.append({
            "tx_id": tx.tx_id,
            "rx_id": 0,
            "distance_km": 0.0,
            "geometry": Point(tx.lon, tx.lat),
        })

    # Generate receivers at each distance × azimuth combination
    for d_km in distances_km:
        radius_m = float(d_km) * 1000.0
        for az in azimuths_deg:
            theta = math.radians(float(az))
            dx = radius_m * math.sin(theta)
            dy = radius_m * math.cos(theta)
            rx_utm = Point(tx_pt.x + dx, tx_pt.y + dy)
            
            # Convert back to WGS84 (EPSG:4326)
            rx_ll = gpd.GeoSeries([rx_utm], crs=utm_crs).to_crs("EPSG:4326").iloc[0]
            
            rows.append({
                "tx_id": tx.tx_id,
                "rx_id": rx_id,
                "distance_km": float(d_km),
                "azimuth_deg": float(az),
                "geometry": rx_ll,
            })
            rx_id += 1

    return gpd.GeoDataFrame(rows, geometry="geometry", crs="EPSG:4326")


print("✓ generate_receivers_radial_multi() defined")

✓ generate_receivers_radial_multi() defined


## Generate All Receiver Points

Create all points at once (batch mode).

In [36]:
print("\n" + "="*60)
print("PHASE 2: BATCH POINT GENERATION")
print("="*60)

print(f"\nGenerating receiver points:")
print(f"  Transmitter: ({tx.lat}, {tx.lon})")
print(f"  Max distance: {max_distance_km} km")
print(f"  Distances: {len(distances)} points @ {CONFIG['RECEIVER_GENERATION']['distance_step']} km spacing")
print(f"  Azimuths: {len(azimuths)} angles @ {CONFIG['RECEIVER_GENERATION']['azimuth_step']}° spacing")
print(f"  Expected points: ~{len(distances) * len(azimuths) + 1}")

# Generate all receiver points at once
start = time.time()
receivers_gdf = generate_receivers_radial_multi(
    tx,
    distances,
    azimuths,
    include_tx_point=True
)
elapsed = time.time() - start

print(f"\n✓ Generated {len(receivers_gdf)} receiver points in {elapsed:.3f}s")
print(f"\nGeoDataFrame structure:")
print(f"  Columns: {list(receivers_gdf.columns)}")
print(f"  CRS: {receivers_gdf.crs}")
print(f"\nFirst 5 points:")
print(receivers_gdf.head())
print(f"\nLast 5 points:")
print(receivers_gdf.tail())


PHASE 2: BATCH POINT GENERATION

Generating receiver points:
  Transmitter: (9.345, -13.40694)
  Max distance: 11 km
  Distances: 367 points @ 0.03 km spacing
  Azimuths: 36 angles @ 10° spacing
  Expected points: ~13213

✓ Generated 13213 receiver points in 5.822s

GeoDataFrame structure:
  Columns: ['tx_id', 'rx_id', 'distance_km', 'geometry', 'azimuth_deg']
  CRS: EPSG:4326

First 5 points:
     tx_id  rx_id  distance_km                   geometry  azimuth_deg
0  TX_0001      0         0.00    POINT (-13.40694 9.345)          NaN
1  TX_0001      1         0.03  POINT (-13.40694 9.34527)          0.0
2  TX_0001      2         0.03  POINT (-13.40689 9.34527)         10.0
3  TX_0001      3         0.03  POINT (-13.40685 9.34525)         20.0
4  TX_0001      4         0.03   POINT (-13.4068 9.34523)         30.0

Last 5 points:
         tx_id  rx_id  distance_km                   geometry  azimuth_deg
13208  TX_0001  13208        11.01  POINT (-13.48344 9.40933)        310.0
13209  TX_

## Summary

**Phase 2 Complete**:
- ✓ All receiver points generated in batch mode
- ✓ GeoDataFrame with geometry, distance, azimuth metadata
- ✓ Ready for Phase 3 (batch data extraction)

**Output**: `receivers_gdf` variable with ~13k points ready for Phase 3

**Next**: Run Phase 3 to extract elevation and land cover data for all points at once